In [4]:

import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, LSTM, SpatialDropout1D
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,
                             f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer

from keras import callbacks
from keras.callbacks import CSVLogger


X = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\train1.csv", header=None)
Y = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\label1.csv", header=None)

T = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\test1.csv", header=None)
C = pd.read_csv(r"E:\学习心得\data\cic-ids2018\02-03\label2.csv", header=None)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)

y_train1 = np.array(Y)
y_test1 = np.array(C)
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)
# trainX.shape[0]
#trainX.shape[1]

# reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
X_test = np.reshape(testT, (testT.shape[0], 1, testT.shape[1]))

batch_size = 50

# 1. define the network
model = Sequential()
#model.add(Embedding(10, emb_dim = 2, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.7))
model.add(LSTM(80,input_shape=(1, 76), return_sequences= True )) 
model.add(Dropout(0.3))
model.add(LSTM(80,return_sequences=True))  
model.add(Dropout(0.3))
model.add(LSTM(80, return_sequences=True)) 
model.add(Dropout(0.3))
model.add(LSTM(80, return_sequences=True))  
model.add(Dropout(0.3))
model.add(LSTM(80, return_sequences=True))  
model.add(Dropout(0.3))
model.add(LSTM(80, return_sequences=False))  
model.add(Dropout(0.3))
model.add(Dense(2))# the no. of output classes
model.add(Activation('sigmoid'))

model.summary()

# try using different optimizers and different optimizer configs
#model.compile(loss = 'mean_squared_error', optimizer = 'sgd', metrics = ['accuracy'])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

csv_logger = CSVLogger('Binlstm80.csv',separator=',', append=False)
history= model.fit(X_train, y_train1, batch_size=batch_size, epochs=500, validation_data=(X_test, y_test1),callbacks=[csv_logger])

loss, accuracy = model.evaluate(X_test, y_test1)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
y_pred = model.predict_classes(X_test)
#print(y_pred)
np.savetxt('Binlst80predicted.txt', y_pred, fmt='%01d')

from sklearn.metrics import confusion_matrix
expected = y_test1
predicted = y_pred
cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %fpr)
print("tpr:")
print("%.3f" %tpr)
print("Classification report for %s", model)
print(metrics.classification_report(expected,predicted))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 1, 80)             50240     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1, 80)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 80)             51520     
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 80)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 1, 80)             51520     
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 80)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 1, 80)            

16778/16778 [==============================] - 249s 15ms/step - loss: 0.0068 - accuracy: 0.9983 - val_loss: 0.0058 - val_accuracy: 0.9984ETA - ETA: 2:56 - loss: 0.0072 -  - ETA: 2:56 - loss: 0.0 - ETA:  - ETA: 2:53 - loss: 0.0071 - accura - ETA: 2:52 - loss: 0.0071  - ETA: 2:52 - loss: 0.0 - ETA: 2:50 - loss: 0.0071 - accuracy - ETA: 2:50 - loss: 0.0071 - accu - ETA: 2:49 - loss: 0.007 - ETA: 2:49 - loss: 0.0072  - ETA:  - ETA: 2:37 - loss: 0.0071 - accuracy - ETA: 2:37 - l - ETA: 2:36 - loss: 0.0071 - accuracy: 0.99 - ETA: 2:34 - loss: 0.0070 - accuracy: 0.99 - ETA: 2:34 - loss: 0.0070 - accuracy:  - ETA: 2:33 - loss: 0.0 - ETA: 2:32 - loss: 0.0 - ETA: 2:31 - loss: 0.0070  - ETA: 2:31 - loss: 0.0 - ETA: 2:30 - loss: 0.0070 - accu - ETA: 2:29 - loss: 0.0070 - ac - ETA: 2: - - ETA: 2:25 - loss: 0 - ETA: 2:24 - loss: 0.0070 - accura - ETA: 2:24 - los - ETA: 2:23 - loss: 0.0069 - accuracy:  - ETA: 2:23 - loss: - ETA: 2:22 - loss: 0.0070 - accu - ETA: 2:22 - loss: 0.0070 - accuracy:  - ETA

16778/16778 [==============================] - 257s 15ms/step - loss: 0.0062 - accuracy: 0.9983 - val_loss: 0.0054 - val_accuracy: 0.9987:48 - loss: 0.0055 - accura - ETA: 3:50 - loss: 0.005 - ETA: 3:50 - loss: 0.0061 - accuracy: 0. - ETA: 3:49 - ETA: 3:44 - loss: 0.0056 - accuracy: 0.99 - ETA: 3:44 - loss: 0.0055 - accuracy: 0.99 - ETA: 3:43 - loss: 0.0055  - ETA: 3:37 - loss: - ETA: 3:29 - loss: 0.0056 - accuracy: 0.99 - ETA: 3:29 - loss: 0.0056 -  - ETA: 3:28 - loss: 0.0059 - accuracy: 0. - ETA:  - ETA: 3:23 - loss: 0.0057 - accura - ETA: 3:23 - loss: 0.0 - ETA: 3:22 - loss: 0.0060 - accuracy - ETA: 3:22 - loss: 0.0060  - ETA: 3:21 - - ETA: 3: - ETA: 3:22 - loss: - ETA: 3:23 - loss: 0.0062 -  - ETA: 3:23 - loss: 0.0061 - accura - ETA: 3:22 - - - ETA: 3:11 - - ETA: 3:10 - loss: 0.006 - ETA: 3:09 - loss: 0.0062  - ETA: 3:08 - loss: 0.0062 - accu - ETA: 3:08 - loss: 0.0062 - accuracy - ETA: 3:07 - loss: 0.0063 - accuracy: 0.99 - ETA: 3:07 - loss: 0.0063 - accuracy - ETA: 3:07 - loss: 0

 7539/16778 [============>.................] - ETA: 2:12 - loss: 0.0049 - accuracy: 0.9986 ETA - ETA: 3:14 - loss: 0.0054 - ac - ETA: 3:13 - loss: 0.0054 - accuracy: 0. - E - ETA: 3:12 - ETA: 2: - E - ETA: 2:29 - loss: 0.0056 - ac - ETA: 2:28 - loss: 0.0056  - ETA: 2:26 - ETA: 2:23 - loss: 0.0056  - ETA: 2:23 - loss: 0 - ETA: 2:23 - loss: 0.0056 - accura - ETA: 2:22 - loss: 0.0056 - accuracy: 0. - ETA: 2:22 - loss: 0.0056 - accuracy - ETA: 2:22 - loss: 0.0056 - accuracy: 0.99 - ETA: 2:22 - loss: - ETA: 2:22 - loss: 0.0056  - ETA: 2:21 - loss: 0.0056 - accuracy: 0. - ETA: 2:21 - loss: 0.0 - - ETA: 2:17 - loss: 0.005 - ETA: 2:17 - loss: 0.005 - ETA: 2:09 - loss: 0.0055 -  - ETA: 2:09 - loss: 0 - ETA: 2:08 - los - ETA: 2:06 - loss: 0.0055 - accuracy:  - ETA: 2:06 - loss: 0 - ETA: 2:06 - loss: 0.0055 - accuracy:  - ETA: 2:06 - loss: 0.0055 - accuracy: 0. - ETA: 2:06 - los - ETA: 2:06 - los - ETA: 2:05 - loss: 0 - ETA: 2:04 - ETA: 2:03 - loss: 0.0 - ETA: 2:00 - loss: 0.0054 - ac - ETA: 2:00

16778/16778 [==============================] - 243s 14ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0046 - val_accuracy: 0.9987- loss: 0.0049 - accuracy: 0.99 - ETA: 1:59 - loss: 0.0049 - accuracy: 0.99 - ETA: 1:59 - loss: 0.0049 - accuracy: 0. - ETA:  - ETA: 1:56 - - ETA: 1:55 - loss: 0.0049 - accuracy:  - ETA: 1:55 - loss: 0.0049 - accura - ETA: 1:55 - loss: 0.0049 - accuracy: 0.99 - ETA: 1:55 - los - ETA: 1:54 - loss: 0.0049 - accuracy - ETA: 1:53 - - ETA: 1:52 - loss: 0.0049 - ac - ETA: 1:52 - loss: 0.0049 - accura - ETA: 1:51 - - ETA: 1:50 - loss: 0.0049 - accuracy:  - ETA: 1:50 - loss: 0.0049 - ac - ETA: 1:50 - loss: 0.004 - ETA: 1:49 - loss: 0.0049 - accuracy - ETA: 1:49 - loss: 0.0049 - accu - ETA: 1:48 - loss: 0.0048 - accuracy: 0.99 - - ETA: 1:45 - loss: 0.0048  - ETA: 1:44 - loss: 0.0048 - accuracy: 0. - ETA: 1:44 - loss: 0.0048 - accura - ETA - ETA: 1:42 - loss: 0.0 - ETA: 1:41 - loss: 0.0048 - accu - ETA: 1: - ETA: 1:38 - loss: 0.0048 - accu - ETA: 1:38 - loss: 0

14205/16778 [========================>.....] - ETA: 33s - loss: 0.0049 - accuracy: 0.9987 00 - ETA: 3:57 - loss: 0.0065 - accuracy: 0.9989 - ETA: 3:45 - loss: 0.0047 - accuracy: 0. - ETA - ETA: 3:30 - loss: 0.0060 - accuracy:  - ETA: 3:29 - loss: 0.0057 - accuracy: 0.99 - ETA: 3:29 - loss: 0 - ETA: 3:27 - loss: 0.0054 - accuracy: 0.99 - ETA: 3:27 - loss: 0.0054 - ac - - ETA: 3:24 - loss: 0.0 - ETA: 3:23 - loss: 0.0055 - accu - ETA: 3:23 - loss: 0 - ETA: 3:25 - loss: 0.0054 - accu - ETA: 3:27 - loss: 0.0052 - accuracy - ETA: 3:28 - l - ETA:  - ETA: 3:27 - loss: 0.005 - ETA: 3:27 - loss: 0.0050 -  - ETA: 3:26 - loss: 0.005 - ETA: 3:25 - loss: - ETA: 3:22 - loss: 0.0 - ETA - ETA: 3:20 - loss: 0.0050 - accu - ETA: 3:19 - loss: 0 - ETA: 3:18 - loss: 0.0050 - accuracy: 0. - ETA: 3:18 - loss: 0.0051 - accuracy: 0. - ETA: 3:18 - loss: 0.0051  - ETA:  - ETA: 3:16 - loss: 0.0052 - accuracy: 0.99 - ETA: 3:16 - loss: 0.0052 - accu - ETA: 3:15 - loss: 0.0052  - ETA: 3:15 - loss: 0.0053 - accuracy: 

16778/16778 [==============================] - 233s 14ms/step - loss: 0.0049 - accuracy: 0.9987 - val_loss: 0.0044 - val_accuracy: 0.9988 accuracy: 0.998 - ETA: 31s - loss: 0.0049 - accu - ETA: 30s - loss: 0.0049 - accu - ETA: 29s - loss: 0.0049 - accuracy:  - ETA: 29s - loss: 0.0049 - accur - ETA: 28s - loss: 0.0049 - accuracy: 0.9 - ETA: 28s - loss: 0.0049 - accuracy: 0.99 - ETA: 28s - loss: 0.0049 - accuracy - ETA: 27s - loss: 0.0049 -  - ETA: 26s - loss: 0.0049 - accuracy - ETA: 26s - loss: 0.0049 - accuracy: 0.998 - ETA: 26s - loss: 0.0049 - accuracy: - ETA: 25s - loss: 0.0049 - accuracy: 0 - ETA: 25s - loss: 0.0049 - accuracy: 0 - ETA: 25s - loss: 0.0049 -  - ETA: 24s - loss: 0.0049  - ETA: 23s - loss: 0.0049 - accuracy:  - ETA: 22s - loss: 0.0049 - accuracy: 0.99 - ETA - ETA: 20s - loss: 0.0049 - ac - ETA: 19s - loss: 0.0049 - accu - ETA: 18s - loss: 0.0049 - - ETA: 17s - loss: 0.0049 -  - ETA: 16s - loss: 0.0049 - ac - ETA: 15s - loss: 0.0049 - accuracy: 0.9 - ETA: 15s - loss: 

KeyboardInterrupt: 